# Kubernetes Demo

## Prerequisites

* .NET Core SDK 3.1
* Docker
* Kubernetes

## Web App Demo

### No Container

In [ ]:
dotnet run --project dotnet-app

**URLs**

http://localhost:5000

http://localhost:5000/stop

### Docker

In [ ]:
rm -r dotnet-app/bin
rm -r dotnet-app/obj

In [ ]:
docker build --pull -t fjb4/dotnetapp .

In [ ]:
docker run --rm -it -p 8000:80 fjb4/dotnetapp

**URLs**

http://localhost:8000

http://localhost:8000/stop

## Intro to Kubernetes

### kubectl

In [ ]:
kubectl config get-contexts

In [ ]:
kubectl config set-context docker-desktop

In [ ]:
kubectl cluster-info

In [ ]:
kubectl get node -o wide

In [ ]:
kubectl get all --all-namespaces

### Services

In [ ]:
cat 1-service.yaml

In [ ]:
kubectl apply -f 1-service.yaml

### Pods

In [ ]:
cat 2-pod.yaml

In [ ]:
kubectl apply -f 2-pod.yaml

In [ ]:
kubectl logs dotnetapp-pod

In [ ]:
kubectl get pod dotnetapp-pod -w

**URLs**

http://localhost:8080/stop

In [ ]:
kubectl delete pod dotnetapp-pod

### ReplicaSet

In [42]:
cat 3-replicaset.yaml

apiVersion: apps/v1
kind: ReplicaSet
metadata:
  name: dotnetapp-rep
  labels:
    app: dotnetapp
spec:
  replicas: 3
  selector:
    matchLabels:
      app: dotnetapp
  template:
    metadata:
      labels:
        app: dotnetapp
    spec:
      containers:
      - image: fjb4/dotnetapp
        name: dotnetapp-container
        imagePullPolicy: Always
        ports:
          - containerPort: 80
        env:
          - name: POD_NAME
            valueFrom:
              fieldRef:
                fieldPath: metadata.name
          - name: POD_IP
            valueFrom:
              fieldRef:
                fieldPath: status.podIP


In [43]:
kubectl apply -f 3-replicaset.yaml

replicaset.apps/dotnetapp-rep created


In [44]:
kubectl get all

NAME                      READY   STATUS              RESTARTS   AGE
pod/dotnetapp-rep-85jzb   0/1     ContainerCreating   0          1s
pod/dotnetapp-rep-s4h4k   0/1     ContainerCreating   0          1s
pod/dotnetapp-rep-zrjk9   0/1     ContainerCreating   0          1s

NAME                    TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
service/dotnetapp-svc   LoadBalancer   10.106.174.26   localhost     8080:30989/TCP   143m
service/kubernetes      ClusterIP      10.96.0.1       <none>        443/TCP          19h

NAME                            DESIRED   CURRENT   READY   AGE
replicaset.apps/dotnetapp-rep   3         3         0       1s


In [45]:
kubectl scale --replicas=5 replicaset dotnetapp-rep

replicaset.extensions/dotnetapp-rep scaled


In [46]:
kubectl get pod -w

NAME                  READY   STATUS    RESTARTS   AGE
dotnetapp-rep-2dcrf   1/1     Running   0          8s
dotnetapp-rep-85jzb   1/1     Running   0          13s
dotnetapp-rep-f9m22   1/1     Running   0          8s
dotnetapp-rep-s4h4k   1/1     Running   0          13s
dotnetapp-rep-zrjk9   1/1     Running   0          13s
dotnetapp-rep-f9m22   0/1     Completed   0          12s
dotnetapp-rep-f9m22   1/1     Running     1          13s


**URLs**

http://localhost:8080/stop

http://localhost:8080/

In [48]:
kubectl delete replicaset dotnetapp-rep

replicaset.extensions "dotnetapp-rep" deleted


### Deployments

In [ ]:
cat 4-deployment.yaml

In [49]:
kubectl apply -f 4-deployment.yaml --record

deployment.apps/dotnetapp-dep created


In [50]:
kubectl get all

NAME                                 READY   STATUS              RESTARTS   AGE
pod/dotnetapp-dep-6d78ccb4df-qnsml   0/1     ContainerCreating   0          3s
pod/dotnetapp-dep-6d78ccb4df-w9bk6   0/1     ContainerCreating   0          3s
pod/dotnetapp-dep-6d78ccb4df-wr78j   0/1     ContainerCreating   0          3s

NAME                    TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
service/dotnetapp-svc   LoadBalancer   10.106.174.26   localhost     8080:30989/TCP   146m
service/kubernetes      ClusterIP      10.96.0.1       <none>        443/TCP          19h

NAME                            READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/dotnetapp-dep   0/3     3            0           3s

NAME                                       DESIRED   CURRENT   READY   AGE
replicaset.apps/dotnetapp-dep-6d78ccb4df   3         3         0       3s


In [56]:
kubectl rollout history deployment dotnetapp-dep

deployment.extensions/dotnetapp-dep 
REVISION  CHANGE-CAUSE
2         kubectl apply --filename=5-deployment.yaml --record=true
3         kubectl apply --filename=4-deployment.yaml --record=true



In [52]:
cat 5-deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: dotnetapp-dep
  labels:
    app: dotnetapp
spec:
  replicas: 3
  selector:
    matchLabels:
      app: dotnetapp
  template:
    metadata:
      labels:
        app: dotnetapp
    spec:
      containers:
      - image: nginx
        name: dotnetapp-container
        imagePullPolicy: Always
        ports:
          - containerPort: 80


In [53]:
kubectl apply -f 5-deployment.yaml --record

deployment.apps/dotnetapp-dep configured


**URLs**

http://localhost:8080/

In [62]:
kubectl rollout undo deployment dotnetapp-dep

deployment.extensions/dotnetapp-dep rolled back


**URLs**

http://localhost:8080/

In [64]:
kubectl rollout history deployment dotnetapp-dep

Error from server (NotFound): deployments.extensions "dotnetapp-dep" not found


In [63]:
kubectl delete deployment dotnetapp-dep

deployment.extensions "dotnetapp-dep" deleted
